### WHO ANC ValueSet

- Note we are using value sets with mapping to SNOMED IPS
- Also constraints are not relevant (too detailed for Momcare)
- We do use Measures (downstream)


In [1]:
import fsspec
import polars as pl


systems = ["ICD-10", "ICD-11", "ICF", "ICHI", "LOINC", "SNOMED-CT"]


def parse_conceptmap(system: str) -> pl.DataFrame:
    "Generate flattened mapping tables from WHO ANC conceptmap."

    if system not in systems:
        return None

    with fsspec.open(
        f"https://build.fhir.org/ig/dhes/smart-anc/ConceptMap-{system}.json"
    ) as f:
        df = pl.read_json(f)

    unnest_group = pl.col("group").list.explode().struct.unnest()
    unnest_element = pl.col("element").list.explode().struct.unnest().list.explode()

    return (
        df.select(unnest_group)
        .select(unnest_element)
        .select(
            pl.col(pl.String).name.prefix("who_anc_"),
            pl.lit(system.replace("-", "")).alias("target"),
            pl.col("target").struct.unnest(),
        )
    )

In [2]:
df = pl.concat([parse_conceptmap(system) for system in systems], how="diagonal")

In [3]:
# 735 unique WHO ANC codes
df.select(pl.col("who_anc_code").n_unique())

who_anc_code
u32
735


In [4]:
# Coverage varies widely, SNOMED most complete
# Multiple WHO ANC code can map to the same target code
df.group_by("target").agg(pl.n_unique("who_anc_code", "code"))

target,who_anc_code,code
str,u32,u32
"""SNOMEDCT""",725,430
"""ICD10""",532,188
"""LOINC""",385,145
"""ICF""",100,32
"""ICD11""",550,225
"""ICHI""",163,42


In [5]:
# Oh my goodness, dear WHO, what have you done?
many_to_one = (
    df.group_by("target", "code")
    .agg(pl.count("who_anc_code").alias("count_"))
    .filter(pl.col("count_") > 1)
)
many_to_one.join(df, on=["target", "code"]).sort(["target", "code"])

target,code,count_,who_anc_code,who_anc_display,equivalence,display
str,str,u32,str,str,str,str
"""ICD10""","""A53.9""",2,"""ANC.B9.DE111""","""Syphilis positive""","""equivalent""","""Syphilis, unspecified"""
"""ICD10""","""A53.9""",2,"""ANC.B9.DE108""","""Syphilis positive""","""equivalent""","""Syphilis, unspecified"""
"""ICD10""","""B18.1""",2,"""ANC.B9.DE72""","""Hepatitis B positive""","""equivalent""","""Chronic viral hepatitis B with…"
"""ICD10""","""B18.1""",2,"""ANC.B9.DE75""","""Hepatitis B positive""","""equivalent""","""Chronic viral hepatitis B with…"
"""ICD10""","""B18.2""",2,"""ANC.B9.DE93""","""Hepatitis C positive""","""equivalent""","""Chronic viral hepatitis C"""
…,…,…,…,…,…,…
"""SNOMEDCT""","""84229001""",3,"""ANC.B7.DE53""","""Gets tired easily""","""equivalent""","""Fatigue (finding)"""
"""SNOMEDCT""","""8517006""",2,"""ANC.B7.DE12""","""Recently quit tobacco products""","""equivalent""","""Ex-smoker (finding)"""
"""SNOMEDCT""","""8517006""",2,"""ANC.B6.DE154""","""Recently quit tobacco products""","""equivalent""","""Ex-smoker (finding)"""


In [ ]:
# import erdantic as erd

# erd.create(PatientIPS).draw("erd.svg")

ModuleNotFoundError: No module named 'erdantic'